In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

In [3]:
from agents.dungeon.monster.dungeon_monster_agent import monster_graph
from agents.dungeon.monster.monster_database import MONSTER_DATABASE
from core.game_dto.StatData import StatData
import json

## 1. 테스트 데이터 준비

In [4]:
# 단일 플레이어 스탯
single_player_stat = {
    "hp": 500,
    "strength": 20,
    "dexterity": 15,
    "intelligence": 10,
    "defense": 10,
    "critChance": 15.0,
    "attackSpeed": 1.5,
    "moveSpeed": 400,
    "skillDamageMultiplier": 1.2,
}

# 파티 플레이어 스탯 (3명)
party_stats = [
    {
        "hp": 600,
        "strength": 25,
        "dexterity": 10,
        "intelligence": 5,
        "defense": 15,
        "critChance": 10.0,
        "attackSpeed": 1.3,
        "moveSpeed": 350,
        "skillDamageMultiplier": 1.0,
    },
    {
        "hp": 400,
        "strength": 15,
        "dexterity": 25,
        "intelligence": 10,
        "defense": 8,
        "critChance": 25.0,
        "attackSpeed": 2.0,
        "moveSpeed": 450,
        "skillDamageMultiplier": 1.5,
    },
    {
        "hp": 350,
        "strength": 10,
        "dexterity": 15,
        "intelligence": 30,
        "defense": 5,
        "critChance": 20.0,
        "attackSpeed": 1.8,
        "moveSpeed": 400,
        "skillDamageMultiplier": 2.0,
    },
]

# 던전 데이터 (보스방 포함)
dungeon_data_with_boss = {
    "dungeon_id": 1,
    "floor_count": 3,
    "rooms": [
        {"room_id": 1, "room_type": "start", "position": {"x": 0, "y": 0}},
        {"room_id": 2, "room_type": "monster", "position": {"x": 10, "y": 0}},
        {"room_id": 3, "room_type": "monster", "position": {"x": 20, "y": 0}},
        {"room_id": 4, "room_type": "treasure", "position": {"x": 30, "y": 0}},
        {"room_id": 5, "room_type": "monster", "position": {"x": 40, "y": 0}},
        {"room_id": 6, "room_type": "boss", "position": {"x": 50, "y": 0}},
    ],
}

# 던전 데이터 (보스방 없음)
dungeon_data_no_boss = {
    "dungeon_id": 2,
    "floor_count": 2,
    "rooms": [
        {"room_id": 1, "room_type": "start", "position": {"x": 0, "y": 0}},
        {"room_id": 2, "room_type": "monster", "position": {"x": 10, "y": 0}},
        {"room_id": 3, "room_type": "monster", "position": {"x": 20, "y": 0}},
        {"room_id": 4, "room_type": "treasure", "position": {"x": 30, "y": 0}},
    ],
}

# 던전 플레이어 데이터
dungeon_player_data = {
    "affection": 50,
    "sanity": 80,
    "difficulty_level": "normal",
}

print("✅ 테스트 데이터 준비 완료")

✅ 테스트 데이터 준비 완료


## 2. 테스트 1: 단일 플레이어 + 보스방

In [5]:
print("=" * 60)
print("테스트 1: 단일 플레이어 + 보스방")
print("=" * 60)

state_single = {
    "heroine_stat": single_player_stat,
    "monster_db": MONSTER_DATABASE,
    "dungeon_data": dungeon_data_with_boss,
    "dungeon_player_data": dungeon_player_data,
    "floor": 1,
}

result_single = monster_graph.invoke(state_single)

print("\n📊 결과:")
print(f"전투력: {result_single['combat_score']:.2f}")
print(f"\n난이도 로그:")
print(json.dumps(result_single['difficulty_log'], indent=2, ensure_ascii=False))

print("\n🏰 배치된 던전:")
for room in result_single['filled_dungeon_data']['rooms']:
    room_type = room['room_type']
    monsters = room.get('monsters', [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            print(f"  - {m['monster_name']} (타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f})")

테스트 1: 단일 플레이어 + 보스방
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node] 난이도 배율: 1.00
[llm_strategy_node] 추천 태그: ['Mixed', 'MemoryFragment', 'Ambush']
[llm_strategy_node] 전략 이유: 히로인의 전체 전투 능력에 맞춘 적절한 긴장감을 위해 기본 난이도를 유지하되, 빠른 스켈레톤(압박 역할)과 약한 슬라임(초중반 견제)을 혼합합니다. 드문 비중으로 강한 중장비형 몬스터를 섞어 변수를 주어 전투 몰입도를 높입니다.

[select_monsters_node] 타겟 위협도: 194.00
[select_monsters_node] 배율: 1.00
[select_monsters_node] 선호도 조건: 3개
[select_monsters_node] 회피 조건: ['BossLike', 'ExtremeBurst', 'UnavoidableAOE']
[_select_monsters_by_strategy] 타겟: 194.00, 필터된 몬스터 수: 2
[보스방] 방 6: 던전 보스 배치
[전투방] 방 2: 2마리 배치
[전투방] 방 3: 1마리 배치
[select_monsters_node] 일반 몬스터 수: 3
[select_monsters_node] 보스방 존재: True
[select_monsters_node] 일반 몬스터 위협도: 205.00
[select_monsters_node] 보스 위협도: 2250.00
[select_monsters_node] 총 위협도: 2455.00
[select_monsters_node] 달성률: 1265.5%

📊 결과:
전투력: 194.00

난이도 로그:
{
  "model_used": "gpt-5-mini-2025-08-07",
  "combat_score": 194.0,
  "is_p

## 3. 테스트 2: 멀티 플레이어 (파티) + 보스방

In [6]:
print("=" * 60)
print("테스트 2: 멀티 플레이어 (파티 3명) + 보스방")
print("=" * 60)

state_party = {
    "heroine_stat": party_stats,
    "monster_db": MONSTER_DATABASE,
    "dungeon_data": dungeon_data_with_boss,
    "dungeon_player_data": dungeon_player_data,
    "floor": 2,
}

result_party = monster_graph.invoke(state_party)

print("\n📊 결과:")
print(f"파티 평균 전투력: {result_party['combat_score']:.2f}")
print(f"파티 인원: {result_party['difficulty_log']['player_count']}명")
print(f"\n난이도 로그:")
print(json.dumps(result_party['difficulty_log'], indent=2, ensure_ascii=False))

print("\n🏰 배치된 던전:")
for room in result_party['filled_dungeon_data']['rooms']:
    room_type = room['room_type']
    monsters = room.get('monsters', [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            print(f"  - {m['monster_name']} (타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f})")

테스트 2: 멀티 플레이어 (파티 3명) + 보스방
[calculate_combat_score_node] 파티 평균 전투력: 183.17
[calculate_combat_score_node] 파티 인원: 3명
[llm_strategy_node] 난이도 배율: 1.00
[llm_strategy_node] 추천 태그: ['Mixed', 'Pacing', 'MemoryFragment']
[llm_strategy_node] 전략 이유: 기본적으로 균형 잡힌 긴장감을 목표로 합니다. 약한 슬라임으로 지속적인 압박을, 스켈레톤으로 속도·위협을 더하고 가끔의 고체력 골렘으로 전투의 변수를 제공합니다. (영향은 제공된 히로인 상태({hero_summary})와 층수({floor})에 따라 세부 조정)

[select_monsters_node] 타겟 위협도: 183.17
[select_monsters_node] 배율: 1.00
[select_monsters_node] 선호도 조건: 3개
[select_monsters_node] 회피 조건: ['VeryHighDamage', 'BossLike']
[_select_monsters_by_strategy] 타겟: 183.17, 필터된 몬스터 수: 2
[보스방] 방 6: 던전 보스 배치
[전투방] 방 2: 2마리 배치
[전투방] 방 3: 1마리 배치
[select_monsters_node] 일반 몬스터 수: 3
[select_monsters_node] 보스방 존재: True
[select_monsters_node] 일반 몬스터 위협도: 205.00
[select_monsters_node] 보스 위협도: 2250.00
[select_monsters_node] 총 위협도: 2455.00
[select_monsters_node] 달성률: 1340.3%

📊 결과:
파티 평균 전투력: 183.17
파티 인원: 3명

난이도 로그:
{
  "model_used": "gpt-5-mini-2025-08-07",
  "combat_score": 18

## 4. 테스트 3: 단일 플레이어 + 보스방 없음

In [7]:
print("=" * 60)
print("테스트 3: 단일 플레이어 + 보스방 없음")
print("=" * 60)

state_no_boss = {
    "heroine_stat": single_player_stat,
    "monster_db": MONSTER_DATABASE,
    "dungeon_data": dungeon_data_no_boss,
    "dungeon_player_data": dungeon_player_data,
    "floor": 1,
}

result_no_boss = monster_graph.invoke(state_no_boss)

print("\n📊 결과:")
print(f"전투력: {result_no_boss['combat_score']:.2f}")
print(f"보스방 존재: {result_no_boss['difficulty_log']['has_boss_room']}")
print(f"\n난이도 로그:")
print(json.dumps(result_no_boss['difficulty_log'], indent=2, ensure_ascii=False))

print("\n🏰 배치된 던전:")
for room in result_no_boss['filled_dungeon_data']['rooms']:
    room_type = room['room_type']
    monsters = room.get('monsters', [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            print(f"  - {m['monster_name']} (타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f})")

테스트 3: 단일 플레이어 + 보스방 없음
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node] 난이도 배율: 1.00
[llm_strategy_node] 추천 태그: ['RequireStats']
[llm_strategy_node] 전략 이유: 히로인의 상세 스탯이 필요합니다. HP, STR, DEX, AttackSpeed, CritChance, SkillDamage, 파티 여부 및 현재 체력/정신 상태를 제공해 주세요.

[select_monsters_node] 타겟 위협도: 194.00
[select_monsters_node] 배율: 1.00
[select_monsters_node] 선호도 조건: 0개
[select_monsters_node] 회피 조건: ['MissingHeroData']
[_select_monsters_by_strategy] 타겟: 194.00, 필터된 몬스터 수: 2
[_place_monsters_in_rooms] 경고: 보스방이 없습니다
[전투방] 방 2: 2마리 배치
[전투방] 방 3: 1마리 배치
[select_monsters_node] 일반 몬스터 수: 3
[select_monsters_node] 보스방 존재: False
[select_monsters_node] 일반 몬스터 위협도: 205.00
[select_monsters_node] 총 위협도: 205.00
[select_monsters_node] 달성률: 105.7%

📊 결과:
전투력: 194.00
보스방 존재: False

난이도 로그:
{
  "model_used": "gpt-5-mini-2025-08-07",
  "combat_score": 194.0,
  "is_party": false,
  "player_count": 1,
  "ai_multiplier": 1.0,
  "ai_reasoning": 

## 5. 테스트 4: 높은 전투력 플레이어

In [ ]:
print("=" * 60)
print("테스트 4: 높은 전투력 플레이어")
print("=" * 60)

# 강한 플레이어 스탯
strong_player_stat = {
    "hp": 1000,
    "strength": 50,
    "dexterity": 40,
    "intelligence": 30,
    "defense": 25,
    "critChance": 30.0,
    "attackSpeed": 2.5,
    "moveSpeed": 500,
    "skillDamageMultiplier": 2.0,
}


state_strong = {
    "heroine_stat": strong_player_stat,
    "monster_db": MONSTER_DATABASE,
    "dungeon_data": dungeon_data_with_boss,
    "dungeon_player_data": dungeon_player_data,
    "floor": 5,
}

result_strong = monster_graph.invoke(state_strong)

print("\n📊 결과:")
print(f"전투력: {result_strong['combat_score']:.2f}")
print(f"AI 난이도 배율: {result_strong['difficulty_log']['ai_multiplier']:.2f}")
print(f"타겟 위협도: {result_strong['difficulty_log']['target_threat']:.2f}")
print(f"실제 위협도: {result_strong['difficulty_log']['actual_threat']:.2f}")
print(f"\nAI 추론: {result_strong['difficulty_log']['ai_reasoning']}")

print("\n🏰 배치된 던전:")
for room in result_strong['filled_dungeon_data']['rooms']:
    room_type = room['room_type']
    monsters = room.get('monsters', [])
    if monsters:
        print(f"\n방 #{room['room_id']} ({room_type}):")
        for m in monsters:
            print(f"  - {m['monster_name']} (타입: {m['monster_type']}, 위협도: {m['threat_level']:.2f})")

테스트 4: 높은 전투력 플레이어
[calculate_combat_score_node] 플레이어 전투력: 386.50
[calculate_combat_score_node] HP: 1000, STR: 50, DEX: 40


## 6. 보스방 검증

In [9]:
print("=" * 60)
print("보스방 검증")
print("=" * 60)

# 모든 테스트 결과에서 보스방 확인
test_results = [
    ("테스트 1 (단일)", result_single),
    ("테스트 2 (파티)", result_party),
    ("테스트 3 (보스없음)", result_no_boss),
    ("테스트 4 (강함)", result_strong),
]

for test_name, result in test_results:
    print(f"\n{test_name}:")
    
    boss_rooms = [r for r in result['filled_dungeon_data']['rooms'] if r.get('room_type') == 'boss']
    
    if not boss_rooms:
        print("  ⚠️  보스방 없음")
        continue
    
    for boss_room in boss_rooms:
        monsters = boss_room.get('monsters', [])
        print(f"  보스방 #{boss_room['room_id']}:")
        
        if not monsters:
            print("    ❌ 몬스터가 배치되지 않음!")
        elif len(monsters) > 1:
            print(f"    ❌ 보스가 {len(monsters)}마리 배치됨 (1마리여야 함)!")
        else:
            monster = monsters[0]
            if monster['monster_type'] == 2:
                print(f"    ✅ {monster['monster_name']} (보스) 정상 배치")
            else:
                print(f"    ❌ {monster['monster_name']} (타입: {monster['monster_type']}) - 보스가 아님!")

print("\n" + "=" * 60)
print("✅ 모든 테스트 완료")
print("=" * 60)

보스방 검증

테스트 1 (단일):
  보스방 #6:
    ❌ 몬스터가 배치되지 않음!

테스트 2 (파티):
  보스방 #6:
    ✅ 던전 보스 (보스) 정상 배치

테스트 3 (보스없음):
  ⚠️  보스방 없음

테스트 4 (강함):
  보스방 #6:
    ✅ 던전 보스 (보스) 정상 배치

✅ 모든 테스트 완료
